In [1]:
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [3]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._
kernel.silent(false)

In [4]:
val pensionsDataset = Dataset.fromCache("../cache/pensionsCube.cache")
pensionsDataset.size

pensionsDataset: Dataset = com.github.propi.rdfrules.data.Dataset@7dbd0b96
res3_1: Int = 2696400

In [5]:
val refPeriodLinking = Dataset("../data/linking/refPeriodLinking.ttl")
refPeriodLinking.size

refPeriodLinking: Dataset = com.github.propi.rdfrules.data.Dataset@21d40239
res4_1: Int = 84

In [6]:
val wikidataCSSARefAreaLinking = Dataset("../data/linking/cssaWikidataLinking.ttl")
wikidataCSSARefAreaLinking.size

wikidataCSSARefAreaLinking: Dataset = com.github.propi.rdfrules.data.Dataset@7677e8ce
res5_1: Int = 90

In [7]:
val wikidata = Dataset.fromCache("../cache/wikidata.cache")

wikidata: Dataset = com.github.propi.rdfrules.data.Dataset@4338c947

In [8]:
val dataset = pensionsDataset + refPeriodLinking + wikidata + wikidataCSSARefAreaLinking
dataset.size

dataset: Dataset = com.github.propi.rdfrules.data.Dataset@720e4878
res7_1: Int = 2720984

In [9]:
val index = dataset.index()

index: Index = com.github.propi.rdfrules.index.Index$FromDatasetFullyPreservedIndex@7c84aacc

In [10]:
val uri = (value: String) => TripleItem.Uri(value)
val appliesTo = uri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")
val wdProperty = (value: Int) => uri("http://www.wikidata.org/prop/P" + value)

val cssaDimension = "https://data.cssz.cz/ontology/dimension/"
val cssaRefPeriod = uri(cssaDimension+"refPeriod")
val cssaPensionKind = uri(cssaDimension+"druh-duchodu")
val cssaRefArea = uri(cssaDimension+"refArea")

val constantsAtObject = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object
val constantsOnlyAtObject = RuleConstraint.ConstantsAtPosition(constantsAtObject)

val qbDataSet = uri("http://purl.org/linked-data/cube#dataSet")
val oldAgeTotal = uri("https://data.cssz.cz/resource/pension-kind/PK_old_age_total_S_SI_SRN_ST_SD_2010")
val regionTotalOldAgeTotal = uri("pensions-by-region-total-PK_old_age_total_S_SI_SRN_ST_SD")

// pensionsMeasures
val averageAge = TripleItem.Uri("https://data.cssz.cz/ontology/measure/prumerny-vek")
val pensionersCount = TripleItem.Uri("https://data.cssz.cz/ontology/measure/pocet-duchodcu")
val averagePensionAmount = TripleItem.Uri("https://data.cssz.cz/ontology/measure/prumerna-vyse-duchodu-v-kc")
val oneOfPensionsMeasures = OneOf(averageAge,pensionersCount,averagePensionAmount)

uri: String => TripleItem.Uri = ammonite.$sess.cmd9$Helper$$Lambda$2951/0x0000000840dc4040@6f57b7fe
appliesTo: TripleItem.Uri = LongUri(
  "http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod"
)
wdProperty: Int => TripleItem.Uri = ammonite.$sess.cmd9$Helper$$Lambda$2952/0x0000000840dc5040@7a8da898
cssaDimension: String = "https://data.cssz.cz/ontology/dimension/"
cssaRefPeriod: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/ontology/dimension/refPeriod"
)
cssaPensionKind: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/ontology/dimension/druh-duchodu"
)
cssaRefArea: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/ontology/dimension/refArea"
)
constantsAtObject: RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object.type = Object
constantsOnlyAtObject: RuleConstraint.ConstantsAtPosition = ConstantsAtPosition(
  Object
)
qbDataSet: TripleItem.Uri = LongUri("http://purl.org/linked-data/cube#dataSet")
oldAgeTotal: TripleItem.Uri = LongUri(
  "https://data.cssz.cz/resour

# pensions -> party

In [40]:
val pensionsPartyPattern: RulePattern = (
    AtomPattern(subject = 'c', predicate = cssaRefArea, `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(6), `object` = 'd') &:   
    AtomPattern(subject = 'd', predicate = wdProperty(102), `object` = 'a') &:
    AtomPattern(subject = 'c', predicate = averagePensionAmount, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = cssaPensionKind, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = qbDataSet, `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = cssaRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = appliesTo, `object` = 'b')
    =>:
    AtomPattern(subject = 'a', predicate = wdProperty(102), `object` = AnyConstant)
)

pensionsPartyPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("https://data.cssz.cz/ontology/dimension/refArea")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(0)),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(
        LongUri(
          "https://data.cssz.cz/ontology/measure/prumerna-vyse-duchodu-v-kc"
        )
      ),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("https://data.cssz.cz/ontology/dime

In [41]:
val pensionsPartyPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(10))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(pensionsPartyPattern)

pensionsPartyPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@75d9d23f

In [42]:
val startTimeMillis = System.currentTimeMillis()
val pensionsPartyPatternTaskRuleset = index.mine(pensionsPartyPatternTask)
println("rules: "+pensionsPartyPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-04-19 20:30:54:070 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=10,
WithConstants=true,
ConstantsPosition=Object,
Timeout=600000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?c),Constant(Constant(528884079)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(774364698)),Variable(?d),Any), Mapped(Variable(?d),Constant(Constant(52500897)),Variable(?a),Any), Mapped(Variable(?c),Constant(Constant(1915729891)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(-2096697191)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(624690160)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(1169043154)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(217712947)),Variable(?b),Any)),Some(Mapped(Variable(?a),Constant(Constant(

1206


pensionsPartyPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@538bdaa

In [48]:
// regex matchujici pensions-by-region-total-
pensionsPartyPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 2)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.filterResolved(rr => {
    val datasetName = rr.body.find(_.predicate.hasSameUriAs(qbDataSet)).get.`object`
    Stringifier(datasetName).matches(".*pensions-by-region-total.*")
}).export("../rulesets/pensionsPartyPatternTaskRulesetFiltered.txt")

# pensions -> alignment

In [26]:
val pensionsPartyAlignmentPattern: RulePattern = (
    AtomPattern(subject = 'd', predicate = cssaRefArea, `object` = 'g') &:
    AtomPattern(subject = 'g', predicate = wdProperty(6), `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:   
    AtomPattern(subject = 'e', predicate = wdProperty(102), `object` = 'b') &:
    AtomPattern(subject = 'd', predicate = averagePensionAmount, `object` = AnyConstant) &:
    AtomPattern(subject = 'd', predicate = cssaPensionKind, `object` = AnyConstant) &:
    AtomPattern(subject = 'd', predicate = qbDataSet, `object` = AnyConstant) &:
    AtomPattern(subject = 'd', predicate = cssaRefPeriod, `object` = 'c') &:
    AtomPattern(subject = 'b', predicate = appliesTo, `object` = 'c') &:
    AtomPattern(subject = 'b', predicate = wdProperty(102), `object` = 'a')
    =>:
    AtomPattern(subject = 'a', predicate = wdProperty(1387), `object` = AnyConstant)
)

pensionsPartyAlignmentPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("https://data.cssz.cz/ontology/dimension/refArea")),
      Variable(Variable(6)),
      Any
    ),
    AtomPattern(
      Variable(Variable(6)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(1)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(
        LongUri(
          "https://data.cssz.cz/ontology/measure/prumerna-vyse-duchodu-v-kc"
        )
      ),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("https://data.cssz.cz/onto

In [42]:
val pensionsPartyAlignmentPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(11))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(30))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(pensionsPartyAlignmentPattern)

cmd42.sc:7: not found: value pensionsPartyAlignmentPattern
    .addPattern(pensionsPartyAlignmentPattern)
                ^Compilation Failed

: 

In [42]:
val startTimeMillis = System.currentTimeMillis()
val pensionsPartyAlignmentPatternTaskRuleset = index.mine(pensionsPartyAlignmentPatternTask)
println("rules: "+pensionsPartyAlignmentPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

cmd42.sc:2: not found: value pensionsPartyAlignmentPatternTask
val pensionsPartyAlignmentPatternTaskRuleset = index.mine(pensionsPartyAlignmentPatternTask)
                                                          ^Compilation Failed

: 

In [29]:
pensionsPartyAlignmentPatternTaskRuleset.export("../rulesets/pensionsPartyAlignmentPatternTaskRuleset.txt")

In [47]:
// regex matchujici pensions-by-region-total-
pensionsPartyAlignmentPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 2)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.filterResolved(rr => {
    val datasetName = rr.body.find(_.predicate.hasSameUriAs(qbDataSet)).get.`object`
    Stringifier(datasetName).matches(".*pensions-by-region-total.*")
}).export("../rulesets/pensionsPartyAlignmentPatternTaskRulesetFiltered.txt")

# party -> pensions

In [19]:
val partyPensionsPattern: RulePattern = (
    AtomPattern(subject = 'a', predicate = cssaRefArea, `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:
    AtomPattern(subject = 'e', predicate = wdProperty(6), `object` = 'd') &:   
    AtomPattern(subject = 'd', predicate = wdProperty(102), `object` = 'c') &:
    AtomPattern(subject = 'c', predicate = wdProperty(102), `object` = AnyConstant) &:
    
    AtomPattern(subject = 'c', predicate = appliesTo, `object` = 'b') &:
    
    AtomPattern(subject = 'a', predicate = cssaRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaPensionKind, `object` = AnyConstant) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = AnyConstant)
    =>:
    AtomPattern(subject = 'a', predicate = averagePensionAmount, `object` = AnyConstant)
)

partyPensionsPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("https://data.cssz.cz/ontology/dimension/refArea")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(3)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(2)),
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(
        LongUri("http://kizi.vse.cz/novp19/diploma-thesis/appliesToRefPeriod")
      ),
      Variable(Variab

In [22]:
val partyPensionsPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(10))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(10))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(partyPensionsPattern)

partyPensionsPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@3231e0b5

In [42]:
val startTimeMillis = System.currentTimeMillis()
val partyPensionsPatternTaskRuleset = index.mine(partyPensionsPatternTask)
println("rules: "+partyPensionsPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-04-20 18:19:11:278 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=10,
WithConstants=true,
ConstantsPosition=Object,
Timeout=600000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?a),Constant(Constant(528884079)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(774364698)),Variable(?d),Any), Mapped(Variable(?d),Constant(Constant(52500897)),Variable(?c),Any), Mapped(Variable(?c),Constant(Constant(52500897)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(217712947)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(1169043154)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(-2096697191)),AnyConstant,Any), Mapped(Variable(?a),Constant(Constant(624690160)),AnyConstant,Any)),Some(Mapped(Variable(?a),Constant(Constant(19

rules: 1206
duration: 140s


startTimeMillis: Long = 1618935551278L
partyPensionsPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@343bee31
endTimeMillis: Long = 1618935691698L
durationSeconds: Long = 140L

In [24]:
partyPensionsPatternTaskRuleset.export("../rulesets/partyPensionsPatternTaskRuleset.txt")

In [25]:
partyPensionsPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 2)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.filterResolved(rr => {
    val datasetName = rr.body.find(_.predicate.hasSameUriAs(qbDataSet)).get.`object`
    Stringifier(datasetName).matches(".*pensions-by-region-total.*")
}).export("../rulesets/partyPensionsPatternTaskRulesetFiltered.txt")

# alignment -> pensions

In [35]:
val alignmentPensionsPattern: RulePattern = (
    AtomPattern(subject = 'a', predicate = cssaRefArea, `object` = 'g') &:
    AtomPattern(subject = 'g', predicate = wdProperty(6), `object` = 'f') &:
    AtomPattern(subject = 'f', predicate = wdProperty(6), `object` = 'e') &:   
    AtomPattern(subject = 'e', predicate = wdProperty(102), `object` = 'c') &:
    AtomPattern(subject = 'd', predicate = wdProperty(1387), `object` = AnyConstant) &:
    AtomPattern(subject = 'c', predicate = wdProperty(102), `object` = 'd') &:
    AtomPattern(subject = 'c', predicate = appliesTo, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaRefPeriod, `object` = 'b') &:
    AtomPattern(subject = 'a', predicate = cssaPensionKind, `object` = AnyConstant) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, `object` = AnyConstant)
    =>:
    AtomPattern(subject = 'a', predicate = averagePensionAmount, `object` = AnyConstant)
)

alignmentPensionsPattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("https://data.cssz.cz/ontology/dimension/refArea")),
      Variable(Variable(6)),
      Any
    ),
    AtomPattern(
      Variable(Variable(6)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(5)),
      Any
    ),
    AtomPattern(
      Variable(Variable(5)),
      Constant(LongUri("http://www.wikidata.org/prop/P6")),
      Variable(Variable(4)),
      Any
    ),
    AtomPattern(
      Variable(Variable(4)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(2)),
      Any
    ),
    AtomPattern(
      Variable(Variable(3)),
      Constant(LongUri("http://www.wikidata.org/prop/P1387")),
      AnyConstant,
      Any
    ),
    AtomPattern(
      Variable(Variable(2)),
      Constant(LongUri("http://www.wikidata.org/prop/P102")),
      Variable(Variable(3)),
      Any
    ),
    AtomPatt

In [38]:
val alignmentPensionsPatternTask = Amie()
    .addThreshold(Threshold.MinSupport(1))
    .addThreshold(Threshold.MaxRuleLength(11))
    .addThreshold(Threshold.MinHeadSize(0))
    .addThreshold(Threshold.Timeout(10))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(alignmentPensionsPattern)

alignmentPensionsPatternTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@8daa960

In [39]:
val startTimeMillis = System.currentTimeMillis()
val alignmentPensionsPatternTaskRuleset = index.mine(alignmentPensionsPatternTask)
println("rules: "+alignmentPensionsPatternTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-04-20 18:13:48:874 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=1,
MinHeadCoverage=0.0,
MinSupport=1,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=11,
WithConstants=true,
ConstantsPosition=Object,
Timeout=600000,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?a),Constant(Constant(528884079)),Variable(?g),Any), Mapped(Variable(?g),Constant(Constant(774364698)),Variable(?f),Any), Mapped(Variable(?f),Constant(Constant(774364698)),Variable(?e),Any), Mapped(Variable(?e),Constant(Constant(52500897)),Variable(?c),Any), Mapped(Variable(?d),Constant(Constant(1496784743)),AnyConstant,Any), Mapped(Variable(?c),Constant(Constant(52500897)),Variable(?d),Any), Mapped(Variable(?c),Constant(Constant(217712947)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(1169043154)),Variable(?b),Any), Mapped(Variable(?a),Constant(Constant(-2096697191)),AnyConstant,Any), Mapped(Variable(?a),Constant(Constant(62469

rules: 724
duration: 183s


startTimeMillis: Long = 1618935228873L
alignmentPensionsPatternTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@47e5c74c
endTimeMillis: Long = 1618935412037L
durationSeconds: Long = 183L

In [40]:
alignmentPensionsPatternTaskRuleset.export("../rulesets/alignmentPensionsPatternTaskRuleset.txt")

In [41]:
alignmentPensionsPatternTaskRuleset
.filter(rule => rule.measures(Measure.Support).value > 2)
.computePcaConfidence(0.0)
.sortBy(Measure.PcaConfidence, Measure.Support)
.filterResolved(rr => {
    val datasetName = rr.body.find(_.predicate.hasSameUriAs(qbDataSet)).get.`object`
    Stringifier(datasetName).matches(".*pensions-by-region-total.*")
}).export("../rulesets/alignmentPensionsPatternTaskRulesetFiltered.txt")